<a href="https://colab.research.google.com/github/jsvir/idc/blob/main/idc_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

# "force_all_finite" 관련 FutureWarning만 무시
warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message=".*force_all_finite was renamed to 'ensure_all_finite'.*"
)

In [2]:
with open("cfg/cfg_run.yaml") as f:
    for line in f.readlines():
        print(line.strip())

### Step 1: data preparation

We provide the PBMC-2 dataset in the data directory, unzip it:


In [3]:
!unzip ./data/pbmc_x.zip

### Step 2: run clustering training

We modify the function that plots the clustering to show the plot in the notebook:

In [4]:
import sys
sys.path.append("./idc")
import umap
import torch
import matplotlib.pyplot as plt

def plot_clustering(val_embs_list, cluster_mtx, current_epoch, silhouette, dbi):
    reducer = umap.UMAP(n_neighbors=10, min_dist=0.1, n_components=2, random_state=0)
    embedding = reducer.fit_transform(torch.cat(val_embs_list, dim=0).cpu().numpy())
    plt.figure(figsize=(5, 3))
    plt.scatter(embedding[:, 0], embedding[:, 1], c=cluster_mtx.numpy(), s=20, edgecolor='k')
    plt.title(f'Clustering (UMAP). Epoch: {current_epoch}. Silhouette: {silhouette:0.3f}. DBI: {dbi:0.3f}')
    plt.show()

Now we train the model.
Use these tensorboard commands to show the tensorboard logs in the notebook. Tested on my local machine bit not in Colab.

In [ ]:
# !tensorboard --logdir logs\example --port 6006 &

In [ ]:
# from tensorboard import notebook
# notebook.list() # View open TensorBoard instances
# notebook.display(port=6006, height=1000)

In [ ]:
!ls

In [ ]:
from omegaconf import OmegaConf
import numpy as np
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from run import BaseModule

torch.set_float32_matmul_precision('high')   # 중간 정도 속도/정밀도 균형

cfg = OmegaConf.load("./cfg/cfg_run.yaml")
cfg.filepath_samples = "pbmc_x.npz" # due to colab issues
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
seed=777
cfg.seed = seed
seed_everything(seed)
np.random.seed(seed)
model = BaseModule(cfg)
model.plot_clustering = plot_clustering
logger = TensorBoardLogger("logs", name="example", log_graph=False)
trainer = Trainer(**cfg.trainer, callbacks=[LearningRateMonitor(logging_interval='step')])
trainer.logger = logger
trainer.fit(model)